In [1]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.cm as cm
import rasterio
import rasterio.plot as rplt
from rasterio.merge import merge
from rasterio.mask import mask
from rasterio.windows import Window as rWindow
from rasterio.enums import Resampling
from matplotlib_scalebar.scalebar import ScaleBar


from typing import List, Any
import os
import pyproj
import pandas as pd

In [2]:
import sgl_processor_v03 as sgl # type: ignore
import sgl_utils # type: ignore
import geopandas as gpd
settings = sgl.ProcessorSGLSettings()
settings.rangestart = "20181201"
settings.rangeend = "20190331"
processorSGL = sgl.initialize(settings)
metas = sgl.get_metadata_from_csv(settings.season_folder, ["tile-181", "tile-182", "tile-183"])
coll = sgl.create_collection_from_meta(settings, metas)

X=====================================================
SGLProcessor started for 20181201 to 20190331
X=====================================================
=====> Get metadata from .csv files.
=====> Create collection from csv meta.
     | 62 windows created for collection.


# Vectorize Lake Extent

In [3]:
lakeextents = sgl.get_vectorized_lakeextents(settings.season_folder, coll, print_bool = True)
lakeextents_combined = gpd.GeoDataFrame(pd.concat([l["data"] for l in lakeextents]))  # type: ignore
lakeextents_combined.set_crs(lakeextents[0]["data"].crs)  # type: ignore
print(f"     | {lakeextents_combined.shape[0]:>6d} | with {lakeextents_combined.crs}")

=====> Get vectorized max lake extents.
     |  24978 | tile-181_L8S2_2018-12-01_2019-03-31_10m_maxlake.tif
     |   8800 | tile-182_L8S2_2018-12-01_2019-03-31_10m_maxlake.tif
     |   3993 | tile-183_L8S2_2018-12-01_2019-03-31_10m_maxlake.tif
     |  37771 | with EPSG:3031


In [4]:
fname_lakeextent = "L8S2_" + settings.rangestart + "_" +  settings.rangeend + "_lakeextent.shp"
lakeextents_combined.to_file(os.path.join(settings.season_folder, fname_lakeextent))

d:\Programme\Anaconda\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


# Vectorize Damage

In [12]:
number_bins = 5
dmgs = sgl.get_vectorized_dmgs(settings.dmg_folder, coll, print_bool = True, category_bins = number_bins)
dmgs_combined = gpd.GeoDataFrame(pd.concat([d["data"] for d in dmgs]))  # type: ignore
dmgs_combined.set_crs(dmgs[0]["data"].crs)  # type: ignore
print(f"     | {dmgs_combined.shape[0]:>6d} | with {dmgs_combined.crs}")

=====> Get vectorized dmg.
     |  12088 | tile-181_S1_20181101_20181110_30m_output_10px_dmg.tif
     |  10437 | tile-182_S1_20181101_20181110_30m_output_10px_dmg.tif
     |  19162 | tile-183_S1_20181101_20181110_30m_output_10px_dmg.tif
     |  41687 | with EPSG:3031


In [13]:
fname_dmg = "S1_" + settings.rangestart + "_" +  settings.rangeend + "_dmg_" + str(number_bins) + ".shp"
dmgs_combined.to_file(os.path.join(settings.dmg_folder, fname_dmg))

d:\Programme\Anaconda\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


# Vectorize Crevasses

In [9]:
crevs = sgl.get_vectorized_crevasse(settings, coll, print_bool = True, target = 45, spread = 15)
active_crevs = gpd.GeoDataFrame(pd.concat([d["data active"] for d in crevs]))  # type: ignore
active_crevs.set_crs(crevs[0]["data active"].crs)  # type: ignore
inactive_crevs = gpd.GeoDataFrame(pd.concat([d["data inactive"] for d in crevs]))  # type: ignore
inactive_crevs.set_crs(crevs[0]["data inactive"].crs)   # type: ignore
print(f"     | {active_crevs.shape[0]:>6d} vs {inactive_crevs.shape[0]:>6d} with {inactive_crevs.crs}")

=====> Get vectorized active crevasses for 45° (15°).
     |   1764 |  50439 | tile-181_S1_20181101_20181110_delta-alpha.tif
     |   1069 |  51568 | tile-182_S1_20181101_20181110_delta-alpha.tif
     |    404 |  30480 | tile-183_S1_20181101_20181110_delta-alpha.tif
     |   3237 vs 132487 with EPSG:3031


In [10]:
fname_dmg = "crev_" + settings.rangestart + "_" +  settings.rangeend + "_active_45(15).shp"
active_crevs.to_file(os.path.join(settings.dmg_folder, fname_dmg))

d:\Programme\Anaconda\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [11]:
fname_dmg = "crev_" + settings.rangestart + "_" +  settings.rangeend + "_inactive_45(15).shp"
inactive_crevs.to_file(os.path.join(settings.dmg_folder, fname_dmg))

d:\Programme\Anaconda\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
